In [14]:
import nilearn
import sklearn
from nilearn import plotting
from nilearn import datasets
from nilearn import image
from nilearn.image import mean_img
from nilearn.image import index_img
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix


from sklearn.model_selection import KFold
from nilearn.decoding import Decoder 

from sklearn import svm
from sklearn.svm import SVC
from nilearn.input_data import NiftiMasker
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
import os


In [15]:
# Set variables
ROOT_DATA='/Users/home/Documents/BIDS/'
#ROOT_DATA='/home/brunomiguel/Documents/data/BIDS/'
SUB='sub-0001'
SES='ses-001'
TASK='innerspeech'
RUN='run-02'


# data folder
data_path=os.path.join(ROOT_DATA, SUB, SES)
print('The data is in this folder - ' + data_path)

# project folder
ROOT_PROJECT="/Volumes/LANGLYBACKU/MVPA_Speech_project"


The data is in this folder - /Users/home/Documents/BIDS/sub-0001/ses-001


In [16]:

# Load mask
mask_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_mask.nii.gz')
brain_mask= image.load_img(mask_fn)

# Load data
bs_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_bold_bs.nii.gz')
bs_maps= image.load_img(bs_fn)

# Load labels
labels_fn=os.path.join(SUB + '_' + SES + '_task-' + TASK + '_'+ RUN +'_labels.csv')
col_list = ["0"]
labels=pd.read_csv(labels_fn, usecols=col_list)


In [5]:
bs_maps.shape


(91, 109, 91, 252)

In [9]:
classification_target=labels['0']

target_tags=labels['0'].unique()

print(target_tags)

['Baseline' 'Phrases' 'Syllables' 'Vogals' 'Words']


## Decoder

In [11]:
# Then we define the various classifiers that we use
classifiers = ['svc_l2', 'svc_l1', 'logistic_l1',
               'logistic_l2', 'ridge_classifier']


# Here we compute prediction scores and run time for all these
# classifiers
import time

cv = 3
classifiers_data = {}

for classifier_name in sorted(classifiers):
    print(70 * '_')

    # The decoder has as default score the `roc_auc`
    decoder = Decoder(estimator=classifier_name, mask=brain_mask,
                      standardize=True, cv=cv)
    t0 = time.time()
    decoder.fit(bs_maps, classification_target)

    classifiers_data[classifier_name] = {}
    classifiers_data[classifier_name]['score'] = decoder.cv_scores_

    print("%10s: %.2fs" % (classifier_name, time.time() - t0))
    for target_tag in target_tags:
        print("    %14s vs all -- AUC: %1.2f +- %1.2f" % (
            target_tag,
            np.mean(classifiers_data[classifier_name]['score'][target_tag]),
            np.std(classifiers_data[classifier_name]['score'][target_tag]))
        )

    # Adding the average performance per estimator
    scores = classifiers_data[classifier_name]['score']
    scores['AVERAGE'] = np.mean(list(scores.values()), axis=0)
    classifiers_data[classifier_name]['score'] = scores

______________________________________________________________________
logistic_l1: 17.70s
          Baseline vs all -- AUC: 0.54 +- 0.03
           Phrases vs all -- AUC: 0.54 +- 0.01
         Syllables vs all -- AUC: 0.51 +- 0.02
            Vogals vs all -- AUC: 0.54 +- 0.01
             Words vs all -- AUC: 0.53 +- 0.01
______________________________________________________________________
logistic_l2: 135.81s
          Baseline vs all -- AUC: 0.52 +- 0.01
           Phrases vs all -- AUC: 0.55 +- 0.00
         Syllables vs all -- AUC: 0.49 +- 0.02
            Vogals vs all -- AUC: 0.51 +- 0.00
             Words vs all -- AUC: 0.52 +- 0.03
______________________________________________________________________
ridge_classifier: 10.19s
          Baseline vs all -- AUC: 0.52 +- 0.01
           Phrases vs all -- AUC: 0.55 +- 0.02
         Syllables vs all -- AUC: 0.49 +- 0.02
            Vogals vs all -- AUC: 0.50 +- 0.00
             Words vs all -- AUC: 0.52 +- 0.03
________________

/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinea

    svc_l1: 23.03s
          Baseline vs all -- AUC: 0.53 +- 0.02
           Phrases vs all -- AUC: 0.54 +- 0.01
         Syllables vs all -- AUC: 0.50 +- 0.02
            Vogals vs all -- AUC: 0.55 +- 0.01
             Words vs all -- AUC: 0.53 +- 0.01
______________________________________________________________________


/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinea

    svc_l2: 1534.59s
          Baseline vs all -- AUC: 0.52 +- 0.01
           Phrases vs all -- AUC: 0.54 +- 0.00
         Syllables vs all -- AUC: 0.49 +- 0.02
            Vogals vs all -- AUC: 0.51 +- 0.01
             Words vs all -- AUC: 0.52 +- 0.03


In [18]:
# Then we define the various classifiers that we use
classifiers = ['logistic_l1']


# Here we compute prediction scores and run time for all these
# classifiers
import time

cv = 5
classifiers_data_a = {}

for classifier_name in sorted(classifiers):
    print(70 * '_')

    # The decoder has as default score the `roc_auc`
    decoder = Decoder(estimator=classifier_name,
                      standardize=True, cv=cv)
    t0 = time.time()
    decoder.fit(bs_maps, classification_target)

    classifiers_data_a[classifier_name] = {}
    classifiers_data_a[classifier_name]['score'] = decoder.cv_scores_

    print("%10s: %.2fs" % (classifier_name, time.time() - t0))
    for target_tag in target_tags:
        print("    %14s vs all -- AUC: %1.2f +- %1.2f" % (
            target_tag,
            np.mean(classifiers_data_a[classifier_name]['score'][target_tag]),
            np.std(classifiers_data_a[classifier_name]['score'][target_tag]))
        )

    # Adding the average performance per estimator
    scores = classifiers_data_a[classifier_name]['score']
    scores['AVERAGE'] = np.mean(list(scores.values()), axis=0)
    classifiers_data_a[classifier_name]['score'] = scores

______________________________________________________________________
logistic_l1: 118.20s
          Baseline vs all -- AUC: 0.53 +- 0.02
           Phrases vs all -- AUC: 0.53 +- 0.02
         Syllables vs all -- AUC: 0.47 +- 0.03
            Vogals vs all -- AUC: 0.52 +- 0.02
             Words vs all -- AUC: 0.49 +- 0.03


In [19]:
from nilearn.datasets import fetch_icbm152_brain_gm_mask
brain_mask = fetch_icbm152_brain_gm_mask()

# Then we define the various classifiers that we use
classifiers = ['logistic_l1']


# Here we compute prediction scores and run time for all these
# classifiers
import time

cv = 5
classifiers_data_a = {}

for classifier_name in sorted(classifiers):
    print(70 * '_')

    # The decoder has as default score the `roc_auc`
    decoder = Decoder(estimator=classifier_name,mask=brain_mask,
                      standardize=True, cv=cv)
    t0 = time.time()
    decoder.fit(bs_maps, classification_target)

    classifiers_data_a[classifier_name] = {}
    classifiers_data_a[classifier_name]['score'] = decoder.cv_scores_

    print("%10s: %.2fs" % (classifier_name, time.time() - t0))
    for target_tag in target_tags:
        print("    %14s vs all -- AUC: %1.2f +- %1.2f" % (
            target_tag,
            np.mean(classifiers_data_a[classifier_name]['score'][target_tag]),
            np.std(classifiers_data_a[classifier_name]['score'][target_tag]))
        )

    # Adding the average performance per estimator
    scores = classifiers_data_a[classifier_name]['score']
    scores['AVERAGE'] = np.mean(list(scores.values()), axis=0)
    classifiers_data_a[classifier_name]['score'] = scores

______________________________________________________________________
logistic_l1: 1880.55s
          Baseline vs all -- AUC: 0.51 +- 0.06
           Phrases vs all -- AUC: 0.52 +- 0.08
         Syllables vs all -- AUC: 0.52 +- 0.05
            Vogals vs all -- AUC: 0.51 +- 0.01
             Words vs all -- AUC: 0.52 +- 0.04
